Programación para la ciencia de datos
============================

--- 

Week 6: Archivos e interacción con el sistema
-----------------------------------------------------



# 7. Ejercicios para practicar

A continuación encontraréis un conjunto de problemas que os pueden servir para practicar los conceptos explicados en esta unidad. Os recomendamos que intentéis realizar estos problemas vosotros mismos y que, una vez realizados, comparéis la solución que proponemos con vuestra solución. No dudéis en dirigir todas las dudas que surjan de la resolución de estos ejercicios, o de las soluciones propuestas, al foro del aula.

1. Cread un código que permita monitorizar el consumo de memoria RAM de la máquina en la que se ejecute. El código guardará los datos de la memoria total y utilizada del sistema durante un periodo de tiempo, capturando los datos en intervalos periódicos.

Estos datos se guardarán en archivos de texto, utilizando un fichero para los datos capturados en cada momento. Así, dentro de la carpeta de datos, habrá una carpeta para los datos de cada día (que tendrá por nombre el año, el mes y el día, escritos seguidos, por ejemplo, `20200318`). Dentro de la carpeta de cada día, habrá un archivo para cada instante de tiempo en el que se hayan obtenido datos (que tendrá por nombre la hora, el minuto y el segundo, separados por guiones bajos, por ejemplo, `14_45_55`). El contenido del archivo serán los dos valores (memoria total y utilizada) separados por comas (por ejemplo, `15571, 4242`).

Cread también el código que permita recuperar todos los datos almacenados y obtener una descripción estadística básica (media, mediana y desviación estándar).

Para ello, implementaremos una serie de funciones que se detallan a continuación.

1.1. Cread una función que reciba como parámetro el nombre de una carpeta (que será `mem_data` por defecto) y cree las carpetas necesarias para almacenar datos para el día actual. Es decir, el código deberá crear, si no existe ya, una carpeta de datos con el nombre que ha recibido como parámetro (o usar `mem_data` si no se ha especificado ningún nombre), y otra carpeta dentro de esta que tenga por nombre el día actual (en el formato año de 4 cifras, mes de 2 cifras, día de 2 cifras, seguidos sin separadores, por ejemplo, `20200318`).

In [1]:
import os
import datetime
import time
import statistics
import pathlib
import psutil
import shutil
import humanize


def crea_carpetas(carpeta_datos='mem_data'):
   '''Pide como argumento-key, pero no requiere, una string'''
   hoy = datetime.datetime.today()
   nombre_carpeta = hoy.strftime('%Y%m%d')
   carpeta_actual = pathlib.Path().resolve()
   
   if not os.path.exists(os.path.join(carpeta_actual, carpeta_datos)):
      os.mkdir(os.path.join(carpeta_actual, carpeta_datos))
   if not os.path.exists(os.path.join(carpeta_datos, nombre_carpeta)):
      os.mkdir(os.path.join(carpeta_datos, nombre_carpeta))

   return os.path.join(carpeta_datos, nombre_carpeta)


carpeta_fecha = crea_carpetas()


1.2. Implementad una función que reciba como parámetro el *path* con la carpeta de la fecha actual (que se ha creado en el apartado anterior) y escriba un fichero con los datos de consumo de memoria del sistema actuales. El archivo debe tener por nombre la hora actual (en el formato `hora_minuto_segundo`, con los ítems separados por guiones bajos, por ejemplo,` 14_45_55`). El contenido del archivo serán los dos valores (memoria total y utilizada) en megabytes separados por comas (por ejemplo, `15571, 4242`).

Para obtener los datos del consumo de memoria, recordad que podéis ejecutar comandos del sistema con el módulo `subprocess` (seguramente necesitaréis buscar información sobre cómo obtener estos datos con comandos de *unix*).

In [2]:
def crea_datos_consumo(carpeta_hoy):
    '''Requiere como argumento posicional la ruta de una carpeta'''
    hoy = datetime.datetime.today()
    hora_actual = hoy.strftime('%H_%M_%S')
    memorias = psutil.virtual_memory()
    memoria_total = float(humanize.naturalsize(memorias[0], binary=True, format = "%.2f").split()[0]) * 1024
    memoria_usada = float(humanize.naturalsize(memorias[3], binary=True, format = "%.2f").split()[0]) * 1024
    with open(os.path.join(carpeta_hoy, hora_actual), 'w') as file:
        file.write(f'{memoria_total}, {memoria_usada}')


1.3. Implementad una función que reciba como parámetros el número de muestras que capturar y el intervalo de tiempo entre cada una de las muestras (en segundos), y que capture los datos del consumo de memoria tantas veces como se haya especificado, esperando el tiempo indicado entre capturas. La función hará uso de las dos funciones definidas anteriormente.


In [3]:
def da_memoria_rafaga(cantidad, tiempo):
    '''Requiere como argumentos posicionales dos números int'''
    for _ in range(cantidad):
        crea_datos_consumo(carpeta_fecha)
        time.sleep(tiempo)


1.4. Llamad a la función definida en el apartado 1.3 y capturad 20 muestras de consumo de memoria, utilizando un intervalo de 3 segundos entre cada captura.


In [4]:
da_memoria_rafaga(20, 3)


1.5. Implementad una función que lea todos los datos que se han capturado, almacenados en una carpeta que recibirá como parámetro (y que, de nuevo, tomará como valor por defecto `mem_data`), y que muestre los siguientes datos:
* El número de muestras leídas.
* La media de la memoria total y utilizada.
* La mediana de la memoria total y utilizada.
* La desviación estándar de la memoria total y utilizada.
* La fecha y hora de la primera y última capturas de las que tenemos datos.

Llamad a la función anterior para obtener un resumen de los datos capturados.

In [5]:
def resume_datos(carpeta_datos='mem_data'):
   '''Pide como argumento-key, pero no requiere, el nombre de la carpeta con los datos'''
   carpeta_actual = pathlib.Path().resolve()
   ruta_carpeta = os.path.join(os.path.join(carpeta_actual, carpeta_datos))
   muestras = []
   
   with os.scandir(ruta_carpeta) as dir_list:
      for elemento in dir_list:
         if os.path.isdir(elemento):
            with os.scandir(elemento) as dir_list:
               for muestra in dir_list:
                  with open(muestra, 'r') as file:
                     muestras.append(file.read().replace(',','').split() + [os.path.basename(elemento.name)] + [os.path.basename(file.name)])

      muestras_total = [float(item[0]) for item in muestras]
      muestras_usada = [float(item[1]) for item in muestras]

      primera_captura_fecha = datetime.datetime.strptime(muestras[0][2], "%Y%m%d").date()
      primera_captura_hora = datetime.datetime.strptime(muestras[0][3], "%H_%M_%S").time()
      primera_captura = datetime.datetime.combine(primera_captura_fecha, primera_captura_hora)

      ultima_captura_fecha = datetime.datetime.strptime(muestras[-1][2], "%Y%m%d").date()
      ultima_captura_hora = datetime.datetime.strptime(muestras[-1][3], "%H_%M_%S").time()
      ultima_captura = datetime.datetime.combine(ultima_captura_fecha, ultima_captura_hora)
      
      print(f'Muestras leídas: {len(muestras)}')
      print(f'Primera captura: {primera_captura}')
      print(f'Última captura: {ultima_captura}\n')
      print(f'Medias: Memoria total, {statistics.mean(muestras_total)}; memoria usada, {statistics.mean(muestras_usada)}')
      print(f'Medianas: Memoria total, {statistics.median(muestras_total)}; memoria usada, {statistics.median(muestras_usada)}')
      print(f'Desviaciones estándar: Memoria total, {statistics.stdev(muestras_total)}; memoria usada, {statistics.stdev(muestras_usada)}')

            
resume_datos()


Muestras leídas: 20
Primera captura: 2022-11-03 17:26:21
Última captura: 2022-11-03 17:27:18

Medias: Memoria total, 12226.56; memoria usada, 5586.944
Medianas: Memoria total, 12226.56; memoria usada, 5591.04
Desviaciones estándar: Memoria total, 0.0; memoria usada, 16.41092524412976


1.6. Implementad una función que cree un archivo comprimido con todos los datos almacenados para cada día. La función recibirá como argumento el nombre de la carpeta de datos (por defecto, `mem_data`) y creará tantos ficheros comprimidos como días de los que disponemos datos. Cada archivo comprimido contendrá todos los archivos de datos de ese día.

Llamad a la función anterior y comprobad que se generan los ficheros comprimidos correctamente.


In [6]:
def crea_zip(carpeta_datos='mem_data'):
   '''Pide como argumento-key, pero no requiere, el nombre de la carpeta con los datos'''
   carpeta_actual = pathlib.Path().resolve()
   ruta_carpeta = os.path.join(os.path.join(carpeta_actual, carpeta_datos))

   with os.scandir(ruta_carpeta) as dir_list:
      for elemento in dir_list:
         if os.path.isdir(elemento):
            shutil.make_archive(elemento, 'zip', elemento)
            shutil.rmtree(elemento) # Elimina la carpeta original una vez creado el zip para que quede más limpio

crea_zip()


# 8. Bibliografía


## 8.1. Bibliografía básica

La codificación es uno de los detalles importantes que se debe considerar cuando hay que leer y/o escribir un archivo y, a menudo, es el origen de dolores de cabeza en muchos programadores (sobre todo en lenguajes de más bajo nivel que Python). Para entender qué es la codificación de caracteres, conocer cuáles son las codificaciones de caracteres más habituales y saber cómo gestiona Python 3 la codificación, leed ahora la [guía de este enlace](https://realpython.com/python-encodings-guide/#python-3-all-in-on-unicode).


## 8.2. Bibliografía adicional (ampliación de conocimientos)

Esta unidad presenta una introducción a cómo interactuar con el sistema de archivos y, en general, con el sistema operativo, desde Python. Así, como introducción, presenta algunas cuestiones de manera inicial y abre la puerta a explorarlas con más detalle. A continuación se listan algunos enlaces que os servirán para seguir explorando algunos de los temas que trabajamos en la unidad, ya sean puramente de programación en Python como del sistema operativo:

* **El sistema de ficheros de Linux**: en la unidad hablamos de interactuar con el sistema de archivos de Linux, pero no entramos a explicar cómo es este sistema de ficheros. Si deseáis leer una introducción a este sistema, este [*Overview*](https://tldp.org/LDP/intro-linux/html/sect_03_01.html) os puede resultar muy útil.

* **Permisos sobre los ficheros en unix**: si tenéis curiosidad por saber cómo funcionan los bits de permiso de los ficheros en unix, os recomendamos leer las tres partes de la serie de artículos sobre los permisos ([1](http://www.filepermissions.com/articles/what-are-file-permissions-in-linux-and-mac), [2](http://www.filepermissions.com/articles/understanding-octal-file-permissions ) y [3](http://www.filepermissions.com/articles/sticky-bit-suid-and-sgid)).

* **Apertura de archivos desde Python**: la función `open` acepta otros argumentos opcionales que no hemos presentado, y que gestionan detalles como el *buffering* de datos, la codificación, la gestión de los errores, la gestión del salto de línea, etc. El lector interesado puede consultar la [documentación oficial de la función `open`](https://docs.python.org/3/library/functions.html#open) para descubrir cómo funcionan estos argumentos y qué opciones se encuentran disponibles.

* **Compresión de archivos**: existen otros formatos de compresión de datos aparte de los que hemos visto en esta unidad. El lector interesado puede leer la documentación del módulo [`gzip`](https://docs.python.org/3/library/gzip.html) para conocer las funciones que permiten trabajar con archivos gzip desde Python.

* **Lectura de ficheros con pandas**: más allá de los ficheros csv, hay otros formatos que también se utilizan a menudo para intercambiar o guardar datos. Pandas dispone de varias funciones para cargar datos provenientes de los formatos de datos más populares, tales como json ([`read_json`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html#pandas.read_json)) o excel ([`read_excel`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html#pandas.read_excel)).

También os recomendamos revisar la documentación oficial de las funciones y clases descritas en esta unidad, que encontraréis enlazadas en cada uno de los apartados que las describen, con el fin de conocer qué parámetros permiten ajustar su funcionamiento.